In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta


In [2]:

handler = TA_Handler(
    symbol="EURUSD",
    exchange="FX_IDC",
    screener="forex",
    interval="1m",
    timeout=None
)
analysis = handler.get_analysis()
handler.get_indicators()
analysis.oscillators
handler.get_indicators()

{'Recommend.Other': 0,
 'Recommend.All': 0.32692308,
 'Recommend.MA': 0.65384615,
 'RSI': 53.77319533,
 'RSI[1]': 53.44317051,
 'Stoch.K': 71.51515152,
 'Stoch.D': 67.07070707,
 'Stoch.K[1]': 66.36363636,
 'Stoch.D[1]': 63.23232323,
 'CCI20': 81.52887139,
 'CCI20[1]': 26.86615186,
 'ADX': 13.22760688,
 'ADX+DI': 12.96217933,
 'ADX-DI': 14.21506447,
 'ADX+DI[1]': 12.99233529,
 'ADX-DI[1]': 14.24813522,
 'AO': -0.00012015,
 'AO[1]': -0.00016644,
 'Mom': 0.00047,
 'Mom[1]': 0.00052,
 'MACD.macd': -2.291e-05,
 'MACD.signal': -4.906e-05,
 'Rec.Stoch.RSI': 0,
 'Stoch.RSI.K': 75.11810378,
 'Rec.WR': 0,
 'W.R': -23.63636364,
 'Rec.BBPower': 0,
 'BBPower': 0.0002462,
 'Rec.UO': 0,
 'UO': 61.00552221,
 'close': 1.08584,
 'EMA5': 1.08576141,
 'SMA5': 1.085752,
 'EMA10': 1.08572008,
 'SMA10': 1.085682,
 'EMA20': 1.08572901,
 'SMA20': 1.0856935,
 'EMA30': 1.0857445,
 'SMA30': 1.08579033,
 'EMA50': 1.0857562,
 'SMA50': 1.0858232,
 'EMA100': 1.08582921,
 'SMA100': 1.085726,
 'EMA200': 1.08603156,
 'S

In [3]:
df = pd.read_csv('EURUSD_M1_202301251915_202305031504.csv',sep = '\t')

df

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2023.01.25,19:15:00,1.08991,1.09012,1.08991,1.09004,60,0,0
1,2023.01.25,19:16:00,1.09004,1.09004,1.08982,1.08982,60,0,0
2,2023.01.25,19:17:00,1.08982,1.08988,1.08981,1.08984,65,0,0
3,2023.01.25,19:18:00,1.08984,1.08993,1.08983,1.08986,37,0,0
4,2023.01.25,19:19:00,1.08988,1.08991,1.08976,1.08981,48,0,0
...,...,...,...,...,...,...,...,...,...
99995,2023.05.03,15:00:00,1.10396,1.10414,1.10394,1.10398,111,0,0
99996,2023.05.03,15:01:00,1.10400,1.10417,1.10387,1.10415,109,0,0
99997,2023.05.03,15:02:00,1.10415,1.10418,1.10388,1.10389,100,0,0
99998,2023.05.03,15:03:00,1.10389,1.10395,1.10380,1.10387,63,0,0


In [4]:
df.rename(columns={'<DATE>': 'Date',
                   '<TIME>': 'Time',
                   '<OPEN>': 'Open',
                   '<HIGH>': 'High',
                   '<LOW>': 'Low',
                   '<CLOSE>': 'Close',
                   '<TICKVOL>': 'Volume'},
          inplace=True, errors='raise')

df = df[['Date','Time','Open','High','Low','Close','Volume']]

df

,Date,Time,Open,High,Low,Close,Volume
0,2023.01.25,19:15:00,1.08991,1.09012,1.08991,1.09004,60
1,2023.01.25,19:16:00,1.09004,1.09004,1.08982,1.08982,60
2,2023.01.25,19:17:00,1.08982,1.08988,1.08981,1.08984,65
3,2023.01.25,19:18:00,1.08984,1.08993,1.08983,1.08986,37
4,2023.01.25,19:19:00,1.08988,1.08991,1.08976,1.08981,48
...,...,...,...,...,...,...,...
99995,2023.05.03,15:00:00,1.10396,1.10414,1.10394,1.10398,111
99996,2023.05.03,15:01:00,1.10400,1.10417,1.10387,1.10415,109
99997,2023.05.03,15:02:00,1.10415,1.10418,1.10388,1.10389,100
99998,2023.05.03,15:03:00,1.10389,1.10395,1.10380,1.10387,63


In [5]:
help(df.ta.indicators())

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [6]:
# bbands on the internet
help(ta.bbands)
df.ta.bbands(length=30, std=2, mamode="sma", ddof=2, append=True)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, ddof=0, mamode=None, talib=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator by John Bollinger.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=5, std=2, mamode="sma", ddof=0
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length, ddof)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
        PERCENT = (close - LOWER) / (UPPER - LOWER)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 5
        std (int): The long 

C:\Users\irantech24\AppData\Local\Temp\ipykernel_21800\2895954176.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ta.bbands(length=30, std=2, mamode="sma", ddof=2, append=True)


,BBL_30_2.0,BBM_30_2.0,BBU_30_2.0,BBB_30_2.0,BBP_30_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
99995,1.103350,1.103787,1.104225,0.079274,0.720187
99996,1.103341,1.103794,1.104248,0.082168,0.892151
99997,1.103343,1.103798,1.104253,0.082395,0.601158
99998,1.103346,1.103801,1.104256,0.082516,0.575756


In [7]:
# dedault numbers
help(ta.bbands)
df.ta.bbands(append=True)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, ddof=0, mamode=None, talib=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator by John Bollinger.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=5, std=2, mamode="sma", ddof=0
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length, ddof)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
        PERCENT = (close - LOWER) / (UPPER - LOWER)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 5
        std (int): The long 

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,1.089704,1.089874,1.090044,0.031108,0.311228
...,...,...,...,...,...
99995,1.103804,1.104002,1.104200,0.035824,0.444374
99996,1.103820,1.103996,1.104172,0.031967,0.936373
99997,1.103778,1.103972,1.104166,0.035159,0.288739
99998,1.103772,1.103970,1.104168,0.035869,0.247463


In [8]:
help(ta.macd)
df.ta.macd(fast=12, slow=26, signal=9, append=True)

Help on function macd in module pandas_ta.momentum.macd:

macd(close, fast=None, slow=None, signal=None, talib=None, offset=None, **kwargs)
    Moving Average Convergence Divergence (MACD)
    
    The MACD is a popular indicator to that is used to identify a security's trend.
    While APO and MACD are the same calculation, MACD also returns two more series
    called Signal and Histogram. The Signal is an EMA of MACD and the Histogram is
    the difference of MACD and Signal.
    
    Sources:
        https://www.tradingview.com/wiki/MACD_(Moving_Average_Convergence/Divergence)
        AS Mode: https://tr.tradingview.com/script/YFlKXHnP/
    
    Calculation:
        Default Inputs:
            fast=12, slow=26, signal=9
        EMA = Exponential Moving Average
        MACD = EMA(close, fast) - EMA(close, slow)
        Signal = EMA(MACD, signal)
        Histogram = MACD - Signal
    
        if asmode:
            MACD = MACD - Signal
            Signal = EMA(MACD, signal)
          

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
99995,0.000070,0.000019,0.000051
99996,0.000085,0.000028,0.000057
99997,0.000075,0.000014,0.000061
99998,0.000065,0.000003,0.000062


In [9]:
help(ta.psar)
df.ta.psar(append=True)

Help on function psar in module pandas_ta.trend.psar:

psar(high, low, close=None, af0=None, af=None, max_af=None, offset=None, **kwargs)
    Parabolic Stop and Reverse (psar)
    
    Parabolic Stop and Reverse (PSAR) was developed by J. Wells Wilder, that is used
    to determine trend direction and it's potential reversals in price. PSAR uses a
    trailing stop and reverse method called "SAR," or stop and reverse, to identify
    possible entries and exits. It is also known as SAR.
    
    PSAR indicator typically appears on a chart as a series of dots, either above or
    below an asset's price, depending on the direction the price is moving. A dot is
    placed below the price when it is trending upward, and above the price when it
    is trending downward.
    
    Sources:
        https://www.tradingview.com/pine-script-reference/#fun_sar
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=66&Name=Parabolic
    
    Calculation:
        Default Input

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,NaN,NaN,0.02,0
1,NaN,1.103890,0.04,0
2,NaN,1.103327,0.06,0
3,NaN,1.102516,0.06,0
4,NaN,1.101754,0.08,0
...,...,...,...,...
99995,1.103480,NaN,0.02,0
99996,1.103496,NaN,0.02,0
99997,1.103512,NaN,0.02,0
99998,1.103527,NaN,0.02,0


In [10]:
help(ta.stoch)
df.ta.stoch(append=True)

Help on function stoch in module pandas_ta.momentum.stoch:

stoch(high, low, close, k=None, d=None, smooth_k=None, mamode=None, offset=None, **kwargs)
    Stochastic (STOCH)
    
    The Stochastic Oscillator (STOCH) was developed by George Lane in the 1950's.
    He believed this indicator was a good way to measure momentum because changes in
    momentum precede changes in price.
    
    It is a range-bound oscillator with two lines moving between 0 and 100.
    The first line (%K) displays the current close in relation to the period's
    high/low range. The second line (%D) is a Simple Moving Average of the %K line.
    The most common choices are a 14 period %K and a 3 period SMA for %D.
    
    Sources:
        https://www.tradingview.com/wiki/Stochastic_(STOCH)
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=332&Name=KD_-_Slow
    
    Calculation:
        Default Inputs:
            k=14, d=3, smooth_k=3
        SMA = Simple Moving Average
     

,STOCHk_14_3_3,STOCHd_14_3_3
13,NaN,NaN
14,NaN,NaN
15,9.615385,NaN
16,16.666667,NaN
17,39.743590,22.008547
...,...,...
99995,60.000000,62.721457
99996,70.588235,63.921569
99997,67.843137,66.143791
99998,63.529412,67.320261


In [11]:
# recommended by internet
help(ta.rsi)
df.ta.rsi(length=14, scalar=3, drift=3, append=True)

Help on function rsi in module pandas_ta.momentum.rsi:

rsi(close, length=None, scalar=None, talib=None, drift=None, offset=None, **kwargs)
    Relative Strength Index (RSI)
    
    The Relative Strength Index is popular momentum oscillator used to measure the
    velocity as well as the magnitude of directional price movements.
    
    Sources:
        https://www.tradingview.com/wiki/Relative_Strength_Index_(RSI)
    
    Calculation:
        Default Inputs:
            length=14, scalar=100, drift=1
        ABS = Absolute Value
        RMA = Rolling Moving Average
    
        diff = close.diff(drift)
        positive = diff if diff > 0 else 0
        negative = diff if diff < 0 else 0
    
        pos_avg = RMA(positive, length)
        neg_avg = ABS(RMA(negative, length))
    
        RSI = scalar * pos_avg / (pos_avg + neg_avg)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 14
        scalar (float): How much to magnify.

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
99995    55.790002
99996    60.653564
99997    51.349380
99998    50.705041
99999    48.729539
Name: RSI_14, Length: 100000, dtype: float64

In [12]:
# default number
help(ta.rsi)
df.ta.rsi(append=True)

Help on function rsi in module pandas_ta.momentum.rsi:

rsi(close, length=None, scalar=None, talib=None, drift=None, offset=None, **kwargs)
    Relative Strength Index (RSI)
    
    The Relative Strength Index is popular momentum oscillator used to measure the
    velocity as well as the magnitude of directional price movements.
    
    Sources:
        https://www.tradingview.com/wiki/Relative_Strength_Index_(RSI)
    
    Calculation:
        Default Inputs:
            length=14, scalar=100, drift=1
        ABS = Absolute Value
        RMA = Rolling Moving Average
    
        diff = close.diff(drift)
        positive = diff if diff > 0 else 0
        negative = diff if diff < 0 else 0
    
        pos_avg = RMA(positive, length)
        neg_avg = ABS(RMA(negative, length))
    
        RSI = scalar * pos_avg / (pos_avg + neg_avg)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 14
        scalar (float): How much to magnify.

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
99995    55.790002
99996    60.653564
99997    51.349380
99998    50.705041
99999    48.729539
Name: RSI_14, Length: 100000, dtype: float64

In [13]:
help(ta.ichimoku)
df.ta.ichimoku(append=True)

Help on function ichimoku in module pandas_ta.overlap.ichimoku:

ichimoku(high, low, close, tenkan=None, kijun=None, senkou=None, include_chikou=True, offset=None, **kwargs)
    Ichimoku Kinkō Hyō (ichimoku)
    
    Developed Pre WWII as a forecasting model for financial markets.
    
    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/ichimoku-ich/
    
    Calculation:
        Default Inputs:
            tenkan=9, kijun=26, senkou=52
        MIDPRICE = Midprice
        TENKAN_SEN = MIDPRICE(high, low, close, length=tenkan)
        KIJUN_SEN = MIDPRICE(high, low, close, length=kijun)
        CHIKOU_SPAN = close.shift(-kijun)
    
        SPAN_A = 0.5 * (TENKAN_SEN + KIJUN_SEN)
        SPAN_A = SPAN_A.shift(kijun)
    
        SPAN_B = MIDPRICE(high, low, close, length=senkou)
        SPAN_B = SPAN_B.shift(kijun)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Serie

(          ISA_9    ISB_26     ITS_9    IKS_26   ICS_26
 0           NaN       NaN       NaN       NaN  1.09011
 1           NaN       NaN       NaN       NaN  1.09013
 2           NaN       NaN       NaN       NaN  1.09010
 3           NaN       NaN       NaN       NaN  1.08997
 4           NaN       NaN       NaN       NaN  1.09006
 ...         ...       ...       ...       ...      ...
 99995  1.103743  1.103465  1.103855  1.103815      NaN
 99996  1.103750  1.103465  1.103905  1.103815      NaN
 99997  1.103750  1.103465  1.103940  1.103815      NaN
 99998  1.103700  1.103465  1.104040  1.103815      NaN
 99999  1.103645  1.103465  1.104040  1.103815      NaN
 
 [100000 rows x 5 columns],
            ISA_9    ISB_26
 100000  1.103610  1.103465
 100001  1.103610  1.103465
 100002  1.103582  1.103465
 100003  1.103610  1.103465
 100004  1.103815  1.103670
 100005  1.103815  1.103675
 100006  1.103815  1.103720
 100007  1.103815  1.103755
 100008  1.103815  1.103755
 100009  1.103817 

In [14]:
help(ta.sma)
df.ta.sma(length=5, append=True)
df.ta.sma(length=10, append=True)
df.ta.sma(length=20, append=True)
df.ta.sma(length=30, append=True)
df.ta.sma(length=50, append=True)
df.ta.sma(length=100, append=True)
df.ta.sma(length=200, append=True)

Help on function sma in module pandas_ta.overlap.sma:

sma(close, length=None, talib=None, offset=None, **kwargs)
    Simple Moving Average (SMA)
    
    The Simple Moving Average is the classic moving average that is the equally
    weighted average over n periods.
    
    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/simple-moving-average-sma/
    
    Calculation:
        Default Inputs:
            length=10
        SMA = SUM(close, length) / length
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 10
        talib (bool): If TA Lib is installed and talib is True, Returns the TA Lib
            version. Default: True
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        adjust (bool): Default: True
        presma (bool, optional): If True, uses SMA for initial value.
        fillna (value, optional): pd.DataFrame.fillna(value)
     

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
99995    1.103845
99996    1.103845
99997    1.103844
99998    1.103842
99999    1.103838
Name: SMA_200, Length: 100000, dtype: float64

In [15]:
help(ta.ema)
df.ta.ema(length=5, append=True)
df.ta.ema(length=10, append=True)
df.ta.ema(length=20, append=True)
df.ta.ema(length=30, append=True)
df.ta.ema(length=50, append=True)
df.ta.ema(length=100, append=True)
df.ta.ema(length=200, append=True)

Help on function ema in module pandas_ta.overlap.ema:

ema(close, length=None, talib=None, offset=None, **kwargs)
    Exponential Moving Average (EMA)
    
    The Exponential Moving Average is more responsive moving average compared to the
    Simple Moving Average (SMA).  The weights are determined by alpha which is
    proportional to it's length.  There are several different methods of calculating
    EMA.  One method uses just the standard definition of EMA and another uses the
    SMA to generate the initial value for the rest of the calculation.
    
    Sources:
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
        https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    
    Calculation:
        Default Inputs:
            length=10, adjust=False, sma=True
        if sma:
            sma_nth = close[0:length].sum() / length
            close[:length - 1] = n

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
99995    1.103664
99996    1.103669
99997    1.103671
99998    1.103673
99999    1.103674
Name: EMA_200, Length: 100000, dtype: float64

In [16]:
help(ta.stoch)
df.ta.stoch(k=20, append=True)

Help on function stoch in module pandas_ta.momentum.stoch:

stoch(high, low, close, k=None, d=None, smooth_k=None, mamode=None, offset=None, **kwargs)
    Stochastic (STOCH)
    
    The Stochastic Oscillator (STOCH) was developed by George Lane in the 1950's.
    He believed this indicator was a good way to measure momentum because changes in
    momentum precede changes in price.
    
    It is a range-bound oscillator with two lines moving between 0 and 100.
    The first line (%K) displays the current close in relation to the period's
    high/low range. The second line (%D) is a Simple Moving Average of the %K line.
    The most common choices are a 14 period %K and a 3 period SMA for %D.
    
    Sources:
        https://www.tradingview.com/wiki/Stochastic_(STOCH)
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=332&Name=KD_-_Slow
    
    Calculation:
        Default Inputs:
            k=14, d=3, smooth_k=3
        SMA = Simple Moving Average
     

,STOCHk_20_3_3,STOCHd_20_3_3
19,NaN,NaN
20,NaN,NaN
21,90.179547,NaN
22,87.142857,NaN
23,78.095238,85.139214
...,...,...
99995,59.031256,63.283419
99996,65.944552,62.342013
99997,61.347518,62.107775
99998,59.098874,62.130314


In [17]:
help(ta.cci)
df.ta.cci(lenght=20, append=True)

Help on function cci in module pandas_ta.momentum.cci:

cci(high, low, close, length=None, c=None, talib=None, offset=None, **kwargs)
    Commodity Channel Index (CCI)
    
    Commodity Channel Index is a momentum oscillator used to primarily identify
    overbought and oversold levels relative to a mean.
    
    Sources:
        https://www.tradingview.com/wiki/Commodity_Channel_Index_(CCI)
    
    Calculation:
        Default Inputs:
            length=14, c=0.015
        SMA = Simple Moving Average
        MAD = Mean Absolute Deviation
        tp = typical_price = hlc3 = (high + low + close) / 3
        mean_tp = SMA(tp, length)
        mad_tp = MAD(tp, length)
        CCI = (tp - mean_tp) / (c * mad_tp)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 14
        c (float): Scaling Constant. Default: 0.015
        talib (bool): If TA Lib is

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
99995    80.580581
99996    83.015441
99997    46.092749
99998     2.059099
99999   -19.016153
Name: CCI_14_0.015, Length: 100000, dtype: float64

In [18]:
help(ta.adx) # I did not get it
df.ta.adx()

Help on function adx in module pandas_ta.trend.adx:

adx(high, low, close, length=None, lensig=None, scalar=None, mamode=None, drift=None, offset=None, **kwargs)
    Average Directional Movement (ADX)
    
    Average Directional Movement is meant to quantify trend strength by measuring
    the amount of movement in a single direction.
    
    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/average-directional-movement-adx/
        TA Lib Correlation: >99%
    
    Calculation:
        DMI ADX TREND 2.0 by @TraderR0BERT, NETWORTHIE.COM
            //Created by @TraderR0BERT, NETWORTHIE.COM, last updated 01/26/2016
            //DMI Indicator
            //Resolution input option for higher/lower time frames
            study(title="DMI ADX TREND 2.0", shorttitle="ADX TREND 2.0")
    
            adxlen = input(14, title="ADX Smoothing")
            dilen = input(14, title="DI Length")
            thold = input(20, title="Threshold")
  

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
99995,19.824170,35.793334,18.953197
99996,20.149951,31.951101,19.423382
99997,20.492671,28.985637,17.410670
99998,20.272977,27.453795,19.309122


In [19]:
help(ta.ao)
df.ta.ao(append=True)

Help on function ao in module pandas_ta.momentum.ao:

ao(high, low, fast=None, slow=None, offset=None, **kwargs)
    Awesome Oscillator (AO)
    
    The Awesome Oscillator is an indicator used to measure a security's momentum.
    AO is generally used to affirm trends or to anticipate possible reversals.
    
    Sources:
        https://www.tradingview.com/wiki/Awesome_Oscillator_(AO)
        https://www.ifcm.co.uk/ntx-indicators/awesome-oscillator
    
    Calculation:
        Default Inputs:
            fast=5, slow=34
        SMA = Simple Moving Average
        median = (high + low) / 2
        AO = SMA(median, fast) - SMA(median, slow)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        fast (int): The short period. Default: 5
        slow (int): The long period. Default: 34
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
 

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
99995    0.000212
99996    0.000196
99997    0.000173
99998    0.000157
99999    0.000150
Name: AO_5_34, Length: 100000, dtype: float64

In [20]:
help(ta.mom)
df.ta.mom(append=True)

Help on function mom in module pandas_ta.momentum.mom:

mom(close, length=None, talib=None, offset=None, **kwargs)
    Momentum (MOM)
    
    Momentum is an indicator used to measure a security's speed (or strength) of
    movement.  Or simply the change in price.
    
    Sources:
        http://www.onlinetradingconcepts.com/TechnicalAnalysis/Momentum.html
    
    Calculation:
        Default Inputs:
            length=1
        MOM = close.diff(length)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 1
        talib (bool): If TA Lib is installed and talib is True, Returns the TA Lib
            version. Default: True
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
99995    0.00037
99996    0.00064
99997    0.00033
99998    0.00027
99999   -0.00005
Name: MOM_10, Length: 100000, dtype: float64

In [21]:
help(ta.uo)
df.ta.uo(append=True)

Help on function uo in module pandas_ta.momentum.uo:

uo(high, low, close, fast=None, medium=None, slow=None, fast_w=None, medium_w=None, slow_w=None, talib=None, drift=None, offset=None, **kwargs)
    Ultimate Oscillator (UO)
    
    The Ultimate Oscillator is a momentum indicator over three different
    periods.  It attempts to correct false divergence trading signals.
    
    Sources:
        https://www.tradingview.com/wiki/Ultimate_Oscillator_(UO)
    
    Calculation:
        Default Inputs:
            fast=7, medium=14, slow=28,
            fast_w=4.0, medium_w=2.0, slow_w=1.0, drift=1
        min_low_or_pc  = close.shift(drift).combine(low, min)
        max_high_or_pc = close.shift(drift).combine(high, max)
    
        bp = buying pressure = close - min_low_or_pc
        tr = true range = max_high_or_pc - min_low_or_pc
    
        fast_avg = SUM(bp, fast) / SUM(tr, fast)
        medium_avg = SUM(bp, medium) / SUM(tr, medium)
        slow_avg = SUM(bp, slow) / SUM(tr, slow

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
99995    52.173228
99996    54.725533
99997    46.254628
99998    42.495496
99999    44.750704
Name: UO_7_14_28, Length: 100000, dtype: float64

In [22]:
help(ta.vwma)
df.ta.vwma(append=True)

Help on function vwma in module pandas_ta.overlap.vwma:

vwma(close, volume, length=None, offset=None, **kwargs)
    Volume Weighted Moving Average (VWMA)
    
    Volume Weighted Moving Average.
    
    Sources:
        https://www.motivewave.com/studies/volume_weighted_moving_average.htm
    
    Calculation:
        Default Inputs:
            length=10
        SMA = Simple Moving Average
        pv = close * volume
        VWMA = SMA(pv, length) / SMA(volume, length)
    
    Args:
        close (pd.Series): Series of 'close's
        volume (pd.Series): Series of 'volume's
        length (int): It's period. Default: 10
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
99995    1.103891
99996    1.103943
99997    1.103969
99998    1.103990
99999    1.103989
Name: VWMA_10, Length: 100000, dtype: float64

In [23]:
df.to_csv('test_df.csv', index = False, encoding='utf-8')

In [24]:
col_names = list(df.columns)
print(col_names)

['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'BBL_30_2.0', 'BBM_30_2.0', 'BBU_30_2.0', 'BBB_30_2.0', 'BBP_30_2.0', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0', 'BBP_5_2.0', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'PSARl_0.02_0.2', 'PSARs_0.02_0.2', 'PSARaf_0.02_0.2', 'PSARr_0.02_0.2', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'RSI_14', 'ISA_9', 'ISB_26', 'ITS_9', 'IKS_26', 'ICS_26', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_30', 'SMA_50', 'SMA_100', 'SMA_200', 'EMA_5', 'EMA_10', 'EMA_20', 'EMA_30', 'EMA_50', 'EMA_100', 'EMA_200', 'STOCHk_20_3_3', 'STOCHd_20_3_3', 'CCI_14_0.015', 'AO_5_34', 'MOM_10', 'UO_7_14_28', 'VWMA_10']


In [25]:
df

,Date,Time,Open,High,Low,Close,Volume,BBL_30_2.0,BBM_30_2.0,BBU_30_2.0,...,EMA_50,EMA_100,EMA_200,STOCHk_20_3_3,STOCHd_20_3_3,CCI_14_0.015,AO_5_34,MOM_10,UO_7_14_28,VWMA_10
0,2023.01.25,19:15:00,1.08991,1.09012,1.08991,1.09004,60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023.01.25,19:16:00,1.09004,1.09004,1.08982,1.08982,60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023.01.25,19:17:00,1.08982,1.08988,1.08981,1.08984,65,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023.01.25,19:18:00,1.08984,1.08993,1.08983,1.08986,37,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023.01.25,19:19:00,1.08988,1.08991,1.08976,1.08981,48,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2023.05.03,15:00:00,1.10396,1.10414,1.10394,1.10398,111,1.103350,1.103787,1.104225,...,1.103754,1.103712,1.103664,59.031256,63.283419,80.580581,0.000212,0.00037,52.173228,1.103891
99996,2023.05.03,15:01:00,1.10400,1.10417,1.10387,1.10415,109,1.103341,1.103794,1.104248,...,1.103770,1.103720,1.103669,65.944552,62.342013,83.015441,0.000196,0.00064,54.725533,1.103943
99997,2023.05.03,15:02:00,1.10415,1.10418,1.10388,1.10389,100,1.103343,1.103798,1.104253,...,1.103774,1.103724,1.103671,61.347518,62.107775,46.092749,0.000173,0.00033,46.254628,1.103969
99998,2023.05.03,15:03:00,1.10389,1.10395,1.10380,1.10387,63,1.103346,1.103801,1.104256,...,1.103778,1.103727,1.103673,59.098874,62.130314,2.059099,0.000157,0.00027,42.495496,1.103990


In [40]:
# Cover and go long when the daily closing price crosses below the lower band. Cover and go short when the daily closing price crosses above the upper band.
def bbands_fun(df):
    df.ta.bbands(length=30, std=2, mamode="sma", ddof=0, append=True)
    buy_bbands_diff = df['BBU_30_2.0'] - df['Close']
    sell_bbands_diff = df['BBL_30_2.0'] - df['Close']
    bbands = np.zeros((len(buy_bbands_diff),1))

    for i in range(len(buy_bbands_diff)-1):
        if (sell_bbands_diff[i+1] * sell_bbands_diff[i] < 0) and (df['Close'][i+1]<df['BBL_30_2.0'][i+1]):
            bbands[i+1] = -1
            bbands[i+2] = -1

    for i in range(len(buy_bbands_diff)-1):
        if (buy_bbands_diff[i+1] * buy_bbands_diff[i] < 0) and (df['Close'][i+1]>df['BBU_30_2.0'][i+1]):
            bbands[i+1] = 1
            bbands[i+2] = 1

    if 'sell_bbands_diff' in df.columns.values.tolist():
        df.drop('bbands', axis=1, inplace=True)
        df.drop('buy_bbands_diff', axis=1, inplace=True)
        df.drop('sell_bbands_diff', axis=1, inplace=True)
    else:
        pass

    df.insert(7, 'bbands', bbands)
    df.insert(8, 'buy_bbands_diff', buy_bbands_diff)
    df.insert(9, 'sell_bbands_diff', sell_bbands_diff)


In [47]:
from techiacals_tradingview import *

In [48]:
oscillators_counter, ma_counter = {"BUY": 0, "SELL": 0, "NEUTRAL": 0}, {
        "BUY": 0, "SELL": 0, "NEUTRAL": 0}
computed_oscillators, computed_ma = {}, {}
bbands_fun(df)

for i_0 in range(len(df.index.tolist())-2):
    i = i_0 + 2
    # RSI (14)
    if None not in [df['RSI_14'][i], df['RSI_14'][i-1]]:
        computed_oscillators["RSI_14_sig"] = Compute.RSI(df['RSI_14'][i], df['RSI_14'][i-1])
        oscillators_counter[computed_oscillators["RSI_14_sig"]] += 1

    # Stoch %K
    if None not in [df['STOCHk_14_3_3'][i], df['STOCHd_14_3_3'][i],df['STOCHk_14_3_3'][i-1], df['STOCHd_14_3_3'][i-1]]:
        computed_oscillators["STOCH_14_3_3_sig"] = Compute.Stoch(
            df['STOCHk_14_3_3'][i], df['STOCHd_14_3_3'][i],df['STOCHk_14_3_3'][i-1], df['STOCHd_14_3_3'][i-1])
        oscillators_counter[computed_oscillators["STOCH_14_3_3_sig"]] += 1

    # CCI (20)
    if None not in [df['CCI_14_0.015'][i], df['CCI_14_0.015'][i-1]]:
        computed_oscillators["CCI_14_0.015_sig"] = Compute.CCI20(
            df['CCI_14_0.015'][i], df['CCI_14_0.015'][i-1])
        oscillators_counter[computed_oscillators["CCI_14_0.015_sig"]] += 1

    # AO
    if None not in [df['AO_5_34'][i], df['AO_5_34'][i-1]] and df['AO_5_34'][i-2] != None:
        computed_oscillators["AO_5_34_sig"] = Compute.AO(
            df['AO_5_34'][i], df['AO_5_34'][i-1], df['AO_5_34'][i-2])
        oscillators_counter[computed_oscillators["AO_5_34_sig"]] += 1

    # Mom (10)
    if None not in [df['MOM_10'][i], df['MOM_10'][i-1]]:
        computed_oscillators["MOM_10_sig"] = Compute.Mom(
            df['MOM_10'][i], df['MOM_10'][i-1])
        oscillators_counter[computed_oscillators["MOM_10_sig"]] += 1

    # MACD
    if None not in [df['MACD_12_26_9'][i], df['MACDs_12_26_9'][i-1]]:
        computed_oscillators["MACDs_12_26_9_sig"] = Compute.MACD(
            df['MACD_12_26_9'][i], df['MACDs_12_26_9'][i-1])
        oscillators_counter[computed_oscillators["MACDs_12_26_9_sig"]] += 1

    # MOVING AVERAGES
    ma_list = ["EMA_10", "SMA_10", "EMA_20", "SMA_20", "EMA_30", "SMA_30",
               "EMA_50", "SMA_50", "EMA_100", "SMA_100", "EMA_200", "SMA_200"]
    ma_list_counter = 0
    for index in ma_list:
        if df[index][i] != None:
            computed_ma[ma_list[ma_list_counter]+'_sig'] = Compute.MA(
                df[index][i], df['Close'][i])
            ma_counter[computed_ma[ma_list[ma_list_counter]+'_sig']] += 1
            ma_list_counter += 1
'''
    # ICHIMOKU
    if df[i]['BBP_30_2.0'] != None:
        computed_ma["Ichimoku"] = Compute.Simple(indicators[45])
        ma_counter[computed_ma["Ichimoku"]] += 1

    # VWMA
    if df[i]['VWMA_10'] != None:
        computed_ma["VWMA_10_sig"] = Compute.Simple(indicators[47])
        ma_counter[computed_ma["VWMA"]] += 1

    # UO
    if df[i]['UO_7_14_28'] != None:
        computed_oscillators["UO_7_14_28_sig"] = Compute.Simple(df[i]['UO_7_14_28'])
        oscillators_counter[computed_oscillators["UO_7_14_28_sig"]] += 1

'''


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
30.88235294118223
nan
37.780040733203904
30.88235294118223
36.654506763951
37.780040733203904
47.2350185533262
36.654506763951
55.004864891454794
47.2350185533262
51.595658886627426
55.004864891454794
57.296399896298844
51.595658886627426
54.529925972471375
57.296399896298844
50.891356630309694
54.529925972471375
50.891356630309694
50.891356630309694
48.50279089235196
50.891356630309694
54.133052484047454
48.50279089235196
53.76070740629515
54.133052484047454
54.435735952425254
53.76070740629515
53.18158961399679
54.435735952425254
48.018823917670886
53.18158961399679
51.52717670341475
48.018823917670886
55.149341105072644
51.52717670341475
56.88418779378339
55.149341105072644
54.6093825363101
56.88418779378339
55.75272189888949
54.6093825363101
56.92129631611126
55.75272189888949
59.60224507443319
56.92129631611126
52.55735720196418
59.60224507443319
54.62035703577286
52.55735720196418
56.6

KeyboardInterrupt: 

In [45]:
'''
df.drop('bbands', axis=1, inplace=True)
df.drop('buy_bbands_diff', axis=1, inplace=True)
df.drop('sell_bbands_diff', axis=1, inplace=True)
df.drop( 'macd_h', axis=1, inplace=True)
df.drop('macd_sig', axis=1, inplace=True)
'''

"\ndf.drop('bbands', axis=1, inplace=True)\ndf.drop('buy_bbands_diff', axis=1, inplace=True)\ndf.drop('sell_bbands_diff', axis=1, inplace=True)\ndf.drop( 'macd_h', axis=1, inplace=True)\ndf.drop('macd_sig', axis=1, inplace=True)\n"

In [ ]:
''' out dated : we are using tradingview_ta
# Cover and go long when the daily closing price crosses below the lower band. Cover and go short when the daily closing price crosses above the upper band.

df.ta.bbands(length=30, std=2, mamode="sma", ddof=0, append=True)
buy_bbands_diff = df['BBU_30_2.0'] - df['Close']
sell_bbands_diff = df['BBL_30_2.0'] - df['Close']
bbands = np.zeros((len(buy_bbands_diff),1))

for i in range(len(buy_bbands_diff)-1):
    if (sell_bbands_diff[i+1] * sell_bbands_diff[i] < 0) and (df['Close'][i+1]<df['BBL_30_2.0'][i+1]):
        bbands[i+1] = -1
        bbands[i+2] = -1
        bbands[i+3] = -1

for i in range(len(buy_bbands_diff)-1):
    if (buy_bbands_diff[i+1] * buy_bbands_diff[i] < 0) and (df['Close'][i+1]>df['BBU_30_2.0'][i+1]):
        bbands[i+1] = 1
        bbands[i+2] = 1
        bbands[i+3] = 1

df.insert(7, 'bbands', bbands)
df.insert(8, 'buy_bbands_diff', buy_bbands_diff)
df.insert(9, 'sell_bbands_diff', sell_bbands_diff)

df
'''

In [ ]:
''' out dated : we are using tradingview_ta
df.ta.macd(fast=12, slow=26, signal=9, append=True)
macd_h = df['MACDh_12_26_9']
macd_sig = np.zeros((len(macd_h),1))

for i in range(len(buy_bbands_diff)-3):
    if (macd_h[i+1] * macd_h[i] < 0) and (0>macd_h[i+1]):
        macd_sig[i+1] = -1
        macd_sig[i+2] = -1
        macd_sig[i+3] = -1

for i in range(len(buy_bbands_diff)-3):
    if (macd_h[i+1] * macd_h[i] < 0) and (0<macd_h[i+1]):
        macd_sig[i+1] = 1
        macd_sig[i+2] = 1
        macd_sig[i+3] = 1

df.insert(10, 'macd_h', macd_h, allow_duplicates=True)
df.insert(11, 'macd_sig', macd_sig, allow_duplicates=True)
np.sum(df['macd_sig'])
'''